In [1]:
# import dependencies
from resources.config import strUsername, strPassword, strToken
from simple_salesforce import Salesforce
import pandas as pd

In [2]:
# log in to Salesforce & connect to metadata API
sf = Salesforce(username=strUsername, password=strPassword, security_token=strToken)

In [4]:
# create a dictionary mapping custom objects' IDs to API Names
dfObj = pd.DataFrame(sf.toolingexecute('query/?q=SELECT+Id,+DeveloperName+FROM+CustomObject')['records'])
dfObj = dfObj.drop('attributes', axis=1)
dfObj = dfObj.rename(columns={'Id':'ObjAPIId', 'DeveloperName':'ObjAPIName'})
dfObj = dfObj.set_index('ObjAPIId')
dfObj['ObjAPIName'] = dfObj['ObjAPIName'] + '__c'
dictObj = dfObj['ObjAPIName'].to_dict()

In [8]:
# create a dataframe of all custom fields
dfFld = pd.DataFrame(sf.toolingexecute('query/?q=SELECT+Id,+DeveloperName,+TableEnumOrId+FROM+CustomField')['records'])
dfFld = dfFld.drop('attributes', axis=1)
dfFld['TableEnumOrId'] = dfFld['TableEnumOrId'].replace(dictObj)
dfFld = dfFld.rename(columns={'Id':'RefMetadataComponentId', 'DeveloperName':'FldAPIName', 'TableEnumOrId':'ObjAPIName'})
dfFld = dfFld[['RefMetadataComponentId', 'ObjAPIName', 'FldAPIName']]
dfFld['FldAPIName'] = dfFld['FldAPIName'] + '__c'
dfFld = dfFld.set_index('RefMetadataComponentId')

In [10]:
# create a "Where Used?" dataframe of all custom fields' MetadataComponents
dfWhereUsed = pd.DataFrame(sf.toolingexecute('query/?q=SELECT+RefMetadataComponentId,+MetadataComponentId,+MetadataComponentType,+MetadataComponentName+FROM+MetadataComponentDependency+WHERE+RefMetadataComponentId+LIKE+\'00N%\'')['records'])
dfWhereUsed = dfWhereUsed.drop('attributes', axis=1)
dfWhereUsed = pd.merge(dfWhereUsed, dfFld, on=['RefMetadataComponentId'])
dfWhereUsed = dfWhereUsed[['RefMetadataComponentId', 'ObjAPIName', 'FldAPIName', 'MetadataComponentId', 'MetadataComponentType', 'MetadataComponentName']]
dfWhereUsed = dfWhereUsed.sort_values(['ObjAPIName', 'FldAPIName', 'MetadataComponentType', 'MetadataComponentName'], ascending=True)
dfWhereUsed.to_excel('WhereUsed.xlsx')